## LOGISTIC REGRESSION

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage


%matplotlib inline

In [2]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


In [3]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [4]:
def predict(w, b, X):
    
    m = X.shape[1]
    Y_pred = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X))
    
    for i in range(A.shape[1]):
        
        if A[:,i] <= 0.5:
            Y_pred[:,i] = 0
        else:
            Y_pred[:,i] = 1
            
    return Y_pred

In [5]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    
#     X_train -- (num_px * num_px * 3, m_train)
#     Y_train -- (1, m_train)
#     X_test --  (num_px * num_px * 3, m_test)
#     Y_test --  (1, m_test)
    
    
    #initialize the paramters
    w = np.zeros((X_train.shape[0], 1))
    b = 0
    m = X_train.shape[1]
    
    #optimization using gradient decent
    
    costs = []
    for i in range(num_iterations):
        
        #forward pass
        A = sigmoid(np.dot(w.T, X_train)+b)
        cost = -1/m*(np.sum(Y_train*np.log(A) + (1-Y_train)*np.log(1-A), axis = 1))
                    
        #backward pass - calcualte derivates
        dw = 1/m * np.dot(X_train, (A-Y_train).T)
        db = 1/m*np.sum((A-Y_train))
                    
        #update paramters
        w = w - (learning_rate*dw)
        b = b - (learning_rate*db)
        
        #record cost to plot the learning rate
        if i % 100 == 0:
            costs.append(cost)
            
        # Print the cost every 100 training iterations
#         if print_cost and i % 100 == 0:
#             print ("Cost after iteration %i: %f" %(i, cost))
            
    
    # Predict test/train set examples (≈ 2 lines of code)
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    
    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
        
    return d
        
    
    

In [6]:
#data preprocessing

# Loading the data (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

#shape and dimensions
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

#reshape the training and test inputs to 2 dimensional arrays by flattening
# change from  shape (num_of_images, num_px, num_px, 3) to shape (num_px  ∗ num_px  ∗  3, 1)

train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T

#standardise(for pictures dividing by 255 is fine)
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.



train_set_x shape: (209, 64, 64, 3)
train_set_y shape: (1, 209)
test_set_x shape: (50, 64, 64, 3)
test_set_y shape: (1, 50)


In [7]:
#calling the model
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

train accuracy: 99.04306220095694 %
test accuracy: 70.0 %
